# Scatter plot map for Walcott 
- will use Rathbun scatter plot for reference 
- but will more detailed hovertext that includes specimen family name and year (but dates on file seem to be wrong)
- if there were fewer points or more time, could use mapbox
    - will us go.Scatter in plotly instead 

In [1]:
import pandas as pd 
import plotly.graph_objects as go
from collections import Counter
import numpy as np

### clean dataframe from original xlsx file
- need to create "sub" dataframe with only the columns I need for my scatter plot map 
- can use example from jones scatter plot map 

In [6]:
walcott_df = pd.read_csv('AWHI_Walcott_dataset.csv')
walcott_df.head()

,Barcode,Catalog Number,Order,Family,family_name,Collector(s),year_collected,Country,Province/State,EZID
0,1355660,208828.0,Myrtales,Onagraceae,Evening primroses,"Walcott, M. V.",1900.0,Mexico,Colima,http://n2t.net/ark:/65665/38d98265a-6fb0-4c82-...
1,1160457,99194.0,Solanales,Hydrophyllaceae,Waterleaf family,"Walcott, C. D.",1900.0,United States,Colorado,http://n2t.net/ark:/65665/30a751787-3a18-4f16-...
2,3089925,80416.0,Apiales,Apiaceae,Umbellifers,"Walcott, C. D.",1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3d8b58966-9a8b-4b3b-...
3,3088994,80403.0,Apiales,Apiaceae,Umbellifers,"Walcott, C. D.",1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3c0f352f5-2101-4057-...
4,1805301,27505.0,Asterales,Asteraceae,Daisy family,"Walcott, C. D.",1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3db47f2f9-1144-47d3-...
